In [1]:
import os
import re

import pystac
import pystac.item
import pystac.link
from pystac.provider import ProviderRole
from pystac.extensions.projection import ProjectionExtension

from pystac.extensions.item_assets import ItemAssetsExtension, AssetDefinition

from datetime import datetime, UTC

import rasterio.warp


In [2]:
from constants import *
from item import create_item, get_img_paths

In [15]:
def proj_epsg_from_item_asset(item):
    for asset_key in item.assets:
        asset = item.assets[asset_key].to_dict()
        if 'proj:epsg' in asset.keys():
            return(asset.get('proj:epsg'))

In [4]:
# WORKING_DIR = os.getcwd()
# STAC_DIR = 'stac_tests'
# COLLECTION_ID = 'corine-land-cover-raster'
# COLLECTION_TITLE = 'CORINE Land Cover Raster'
# COLLECTION_DESCRIPTION = ("The European Commission launched the CORINE (Coordination of Information on the Environment) "
#                           "program in an effort to develop a standardized methodology for producing continent-scale land "
#                           "cover, biotope, and air quality maps. The CORINE Land Cover (CLC) product offers a pan-European "
#                           "land cover and land use inventory with 44 thematic classes, ranging from broad forested areas "
#                           "to individual vineyards.")
# COLLECTION_KEYWORDS =  ["clms", "corine", "derived data", "land cover", "machine learning", "open data"]
# COLLECTION_LICENSE = 'proprietary'


# CLMS_CATALOG_LINK = pystac.link.Link(
#     rel=pystac.RelType.ROOT, target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, "stacs/clms_catalog.json"))
# )
# COLLECTION_LINK = pystac.link.Link(
#     rel=pystac.RelType.COLLECTION,
#     target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, f"stacs/{COLLECTION_ID}/{COLLECTION_ID}.json")),
# )
# ITEM_PARENT_LINK = pystac.link.Link(
#     rel=pystac.RelType.PARENT,
#     target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, f"stacs/{COLLECTION_ID}/{COLLECTION_ID}.json")),
# )

In [5]:
data_root = '../CLC_100m'

In [6]:
def create_collection(data_root: str):

    sp_extent = pystac.SpatialExtent([None, None, None, None])
    tmp_extent = pystac.TemporalExtent([datetime(1990, 1, 1, microsecond=0, tzinfo=UTC), None])
    extent = pystac.Extent(sp_extent,  tmp_extent)

    collection = pystac.Collection(id=COLLECTION_ID,
                                description=COLLECTION_DESCRIPTION,
                                title=COLLECTION_TITLE,
                                extent=extent,
                                keywords=COLLECTION_KEYWORDS,
                                license=COLLECTION_LICENSE,
                                stac_extensions=[]
                                )


    item_assets = ItemAssetsExtension.ext(collection, add_if_missing=True)
    item_assets.item_assets = {
        key: AssetDefinition({"title": COLLITAS_TITLE_DICT[key].format(label='').strip(),
                            "media_type": COLLITAS_MEDIA_TYPE_DICT[key], 
                            "roles": COLLITAS_ROLES_DICT[key]})
        for key in COLLITAS_TITLE_DICT
    }

    collection.add_link(CLMS_LICENSE)
    collection.set_self_href(os.path.join(WORKING_DIR, f"{STAC_DIR}/{collection.id}/{collection.id}.json"))
    catalog = pystac.read_file(f"{WORKING_DIR}/{STAC_DIR}/clms_catalog.json")

    collection.set_root(catalog)
    collection.set_parent(catalog)

    collection.save_object()

    img_paths = get_img_paths(path=data_root)

    proj_epsg = []
    for img_path in img_paths:
        item = create_item(img_path, data_root)
        collection.add_item(item)

        item_epsg = proj_epsg_from_item_asset(item)
        proj_epsg.append(item_epsg)

        item.set_self_href(os.path.join(WORKING_DIR, f"{STAC_DIR}/{COLLECTION_ID}/{item.id}/{item.id}.json"))
        item.save_object()

    collection.make_all_asset_hrefs_relative()
    collection.update_extent_from_items()
    ProjectionExtension.add_to(collection)
    collection.summaries = pystac.Summaries({'proj:epsg': list(set(proj_epsg))})

    collection.save_object()